In [114]:
import os
import subprocess
import pandas as pd
import pyarrow.parquet

In [16]:
table_id = "question_time_csv_licensed.zip_02-03_352.csv"
csv_folder_path = "/data3/zecca/projects/sloth/armadillo/gittables/dataset/tables_csv"

table_path = f'{csv_folder_path}/{table_id}'

In [18]:
os.path.exists(csv_folder_path)

False

In [17]:
table = pd.read_csv(table_path)
table

FileNotFoundError: [Errno 2] No such file or directory: '/data3/zecca/projects/sloth/armadillo/gittables/dataset/parquet_tables/0_Sheik_0_tweets.parquet'

In [9]:
table['Content'][3]

'We\'re not trying to compete for attention in the social web. Part of that is reflected in our involvement with the W3C Open Annotation Working Group (<a href="http://www.w3.org/community/openannotation/" rel="nofollow">http://www.w3.org/community/openannotation/</a>). While Hypothes.is may be one place to publish, the goal is to provide better linking and referencing tools to facilitate aggregators and plugins that can bring the conversation back to the source material. In other words, to bring this conversation on HN into the RWW frame and vice versa and to allow better quoting and transclusion between these content silos.'

In [138]:
from tqdm import tqdm
from tools.utils.mongodb_utils import get_mongodb_collections


mongoclient, collections = get_mongodb_collections('gittables', 'standard')

tables_story = []
tables_comment = []

for collection in collections:
    for document in tqdm(collection.find({}), total=collection.count_documents({})):
        content = document['content']
        content = {cell for row in content for cell in row}
        if 'story' in content:
            tables_story.append(document['_id'])
        if 'comment' in content:
            tables_comment.append(document['_id'])


100%|██████████| 759970/759970 [05:30<00:00, 2301.10it/s]


In [139]:
with open('tables_story.txt', 'w') as fs:
    for tabid in tqdm(tables_story):
        fs.write(f'{tabid}\n')

100%|██████████| 296041/296041 [00:00<00:00, 1506231.40it/s]


In [140]:
with open('tables_comment.txt', 'w') as fs:
    for tabid in tqdm(tables_comment):
        fs.write(f'{tabid}\n')

100%|██████████| 295973/295973 [00:00<00:00, 1625044.49it/s]


In [23]:
with open('tables_story.txt', 'r') as fs:
    tables_story = fs.readlines()

In [134]:
with open('tables_comment.txt', 'r') as fs:
    tables_comment = fs.readlines()

In [141]:
len(tables_story), len(tables_comment)

(296041, 295973)

In [142]:
len(set(tables_story).intersection(set(tables_comment)))

292792

In [137]:
tables_comment[:3]

['0\n', '2\n', '3\n']

In [25]:
tables_story[:10]

['question_time_csv_licensed.zip_02-03_352.csv\n',
 'whole_csv_licensed.zip_01-02_411.csv\n',
 'parent_csv_licensed.zip_10-11_968.csv\n',
 'terminal_velocity_csv_licensed.zip_03-04_1.csv\n',
 'living_thing_csv_licensed.zip_02-03_231.csv\n',
 'whole_csv_licensed.zip_07-08_941.csv\n',
 'living_thing_csv_licensed.zip_00-01_331.csv\n',
 'parent_csv_licensed.zip_15-16_600.csv\n',
 'parent_csv_licensed.zip_03-04_88.csv\n',
 'whole_csv_licensed.zip_02-03_1059.csv\n']

In [26]:
all('zip' in s for s in tables_story)

True

In [27]:
tables_story_clean = tables_story
tables_story_clean = [s[:s.find('.zip')] for s in tables_story_clean]

In [28]:
from collections import Counter

freq = Counter(tables_story_clean)

In [29]:
freq

Counter({'parent_csv_licensed': 34367,
         'thing_csv_licensed': 30887,
         'then_csv_licensed': 26664,
         'whole_csv_licensed': 21626,
         'question_time_csv_licensed': 21273,
         'real_time_csv_licensed': 20936,
         'show_time_csv_licensed': 17350,
         'running_time_csv_licensed': 15112,
         'living_thing_csv_licensed': 11602,
         'half_life_csv_licensed': 10183,
         'lead_time_csv_licensed': 9799,
         'data_rate_csv_licensed': 9426,
         'quick_time_csv_licensed': 8834,
         'object_csv_licensed': 7256,
         'processing_time_csv_licensed': 5643,
         'rate_of_return_csv_licensed': 4166,
         'reaction_time_csv_licensed': 3978,
         'growth_rate_csv_licensed': 3501,
         'abstraction_csv_licensed': 2615,
         'speed_of_light_csv_licensed': 2613,
         'bits_per_second_csv_licensed': 2479,
         'id_csv_licensed': 2355,
         'seek_time_csv_licensed': 2219,
         'safe_period_csv_licens

In [144]:
tables_story[:10]

['question_time_csv_licensed.zip_02-03_352.csv',
 'whole_csv_licensed.zip_01-02_411.csv',
 'parent_csv_licensed.zip_10-11_968.csv',
 'terminal_velocity_csv_licensed.zip_03-04_1.csv',
 'living_thing_csv_licensed.zip_02-03_231.csv',
 'whole_csv_licensed.zip_07-08_941.csv',
 'living_thing_csv_licensed.zip_00-01_331.csv',
 'parent_csv_licensed.zip_15-16_600.csv',
 'parent_csv_licensed.zip_03-04_88.csv',
 'whole_csv_licensed.zip_02-03_1059.csv']

In [145]:
import re
pattern = re.compile(r'[0-9]{2}-[0-9]{2}')
x = 0
for table_id in tables_story:
    if pattern.search(table_id):
        x += 1
x

292526

In [147]:
x * 100 / len(tables_story)

98.81266446201708

In [ ]:
from urllib.request import urlretrieve

url = 'https://zenodo.org/records/6517052/files/time_interval_tables_licensed.zip?download=1'
# url = 'https://zenodo.org/records/6517052/files/living_thing_tables_licensed.zip?download=1'
# url = 'https://zenodo.org/records/6517052/files/half_life_tables_licensed.zip?download=1'
zip_path = 'parquet_tables.zip'

urlretrieve(url, zip_path)

('parquet_tables.zip', <http.client.HTTPMessage at 0x7fb575cc3730>)

In [126]:
# topic = 'time_interval'
# topic = 'living_thing'
topic = 'half_life'

In [127]:
matching_tables = [s for s in tables_story if s.startswith(topic)]

In [128]:
matching_tables[:3]

['half_life_csv_licensed.zip_07-08_198.csv\n',
 'half_life_csv_licensed.zip_perpich.csv\n',
 'half_life_csv_licensed.zip_17-18_84.csv\n']

In [129]:
len(matching_tables)

10183

In [130]:
j = 0
for i in range(len(matching_tables)):
    try:
        id = matching_tables[i].strip()[len(f'{topic}_csv_licensed.zip_'):-4]
        table = pyarrow.parquet.read_schema(f'parquet_tables_{topic}/{id}.parquet')
        if 'ali' not in str(table):
            # print(i)
            pass
        else:
            j += 1
    except: continue    
print('j=', j)

j= 10005


In [131]:
j * 100 / len(matching_tables)

98.25198860846508

In [148]:
i = 134
id = matching_tables[i].strip()[len(f'{topic}_csv_licensed.zip_'):-4]
table = pyarrow.parquet.read_schema(f'parquet_tables_{topic}/{id}.parquet')
table.metadata

{b'gittables': b'{"license": "MIT License", "csv_url": "https://github.com/alixaxel/dump.HN/blob/11fd7291c89c68daf4196a47e8cfa47e4df30c8d/data/items/2009/12/22/23-24.csv", "dtypes": {"ID": "int64", "Type": "object", "Story": "float64", "Parent": "float64", "Points": "int64", "Comments": "int64", "Author": "object", "Title": "object", "URL": "object", "Content": "object", "Created": "float64"}, "number_rows": 103, "number_columns": 11, "dtypes_percentages": {"int64": 0.2727272727272727, "object": 0.45454545454545453, "float64": 0.2727272727272727}, "dbpedia_syntactic_column_types": {"Author": {"cleaned_label": "author", "description": null, "domain": ["http://dbpedia.org/ontology/Work"], "id": "http://dbpedia.org/ontology/author", "range": ["http://dbpedia.org/ontology/Person"], "superproperty": ["http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#coparticipatesWith"]}, "Created": {"cleaned_label": "created", "description": null, "domain": ["http://dbpedia.org/ontology/Person"], "id"

In [178]:
import urllib.parse

In [179]:
urllib.parse.urlparse(
    eval(table.metadata[b'gittables'].decode().replace('null', 'None'))['csv_url']
)

ParseResult(scheme='https', netloc='github.com', path='/alixaxel/dump.HN/blob/11fd7291c89c68daf4196a47e8cfa47e4df30c8d/data/items/2009/12/22/23-24.csv', params='', query='', fragment='')

In [168]:
eval('{"a": null}')

NameError: name 'null' is not defined

In [62]:
import polars as pl
df = pl.read_parquet(f'parquet_tables_{topic}/{id}.parquet')
df

ID,Type,Story,Parent,Points,Comments,Author,Title,URL,Content,Created
i64,str,f64,f64,i64,i64,str,str,str,str,f64
523902,"""comment""",523760.0,523882.0,0,0,"""dbul""",null,null,"""My comment shouldn't be taken …",1.2375e9
523901,"""comment""",523280.0,523776.0,0,0,"""zepolen""",null,null,"""Scaling = Increasing capacity …",1.2375e9
523900,"""comment""",523108.0,523108.0,0,0,"""c0un7d0wn""",null,null,"""What are the advantages of thi…",1.2375e9
523899,"""story""",null,null,1,2,"""amichail""","""Do academics exaggerate startu…",null,"""Having people succeed on their…",1.2375e9
523898,"""comment""",523760.0,523891.0,0,0,"""fixie""",null,null,"""Reference for ~700,000 new use…",1.2375e9
…,…,…,…,…,…,…,…,…,…,…
523805,"""comment""",523687.0,523726.0,0,0,"""ntyntyesr""",null,null,"""Which means you wouldn't be re…",1.2375e9
523804,"""comment""",523078.0,523715.0,0,0,"""run4yourlives""",null,null,"""But what's the point of leavin…",1.2375e9
523803,"""story""",null,null,2,0,"""jaspertheghost""","""Harness the power of XML to Op…","""http://www.ibm.com/developerwo…",null,1.2375e9


In [64]:
pq_table = pyarrow.parquet.read_table(f'parquet_tables_{topic}/{id}.parquet')
pq_table.schema.metadata

{b'gittables': b'{"license": "MIT License", "csv_url": "https://github.com/alixaxel/dump.HN/blob/11fd7291c89c68daf4196a47e8cfa47e4df30c8d/data/items/2009/03/19/19-20.csv", "dtypes": {"ID": "int64", "Type": "object", "Story": "float64", "Parent": "float64", "Points": "int64", "Comments": "int64", "Author": "object", "Title": "object", "URL": "object", "Content": "object", "Created": "float64"}, "number_rows": 99, "number_columns": 11, "dtypes_percentages": {"int64": 0.2727272727272727, "object": 0.45454545454545453, "float64": 0.2727272727272727}, "dbpedia_syntactic_column_types": {"Author": {"cleaned_label": "author", "description": null, "domain": ["http://dbpedia.org/ontology/Work"], "id": "http://dbpedia.org/ontology/author", "range": ["http://dbpedia.org/ontology/Person"], "superproperty": ["http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#coparticipatesWith"]}, "Created": {"cleaned_label": "created", "description": null, "domain": ["http://dbpedia.org/ontology/Person"], "id":

In [34]:
pyarrow.parquet.read_schema('time_interval_tables/22-23_1.parquet')

ID: int64
Type: string
Story: double
Parent: double
Points: int64
Comments: int64
Author: string
Title: string
URL: string
Content: string
Created: double
-- schema metadata --
gittables: '{"license": "MIT License", "csv_url": "https://github.com/ali' + 11184
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1471

In [32]:
schema

ID: int64
Type: string
Story: double
Parent: double
Points: int64
Comments: int64
Author: string
Title: string
URL: string
Content: string
Created: double
-- schema metadata --
gittables: '{"license": "MIT License", "csv_url": "https://github.com/ali' + 11184
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1471

In [46]:
import requests

session = requests.Session()

In [47]:
topic = 'marvel'
query = f"https://api.github.com/search/code?q={topic}+in:file+extension:csv&per_page=100"

In [49]:
response = session.get(query)

In [51]:
response.json()

{'message': 'Requires authentication',
 'errors': [{'message': 'Must be authenticated to access the code search API',
   'resource': 'Search',
   'field': 'q',
   'code': 'invalid'}],
 'documentation_url': 'https://docs.github.com/rest/search/search#search-code',
 'status': '401'}